# Keras - LSTM (advanced)

bla

#### Importere biblioteker

In [1]:
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt

# Making plots look better (Jupyter Notebook spesific)
import matplotlib as mpl
mpl.rcParams['patch.force_edgecolor'] = True

# Built in jupyter notebook commands
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#### Importere datasett

In [2]:
data_raw = pd.read_csv('data_ytrevikna_advanced.csv',sep =';', low_memory = False, index_col=0)

data_raw.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 5237 entries, 1.745666504 to 2.072566406
Data columns (total 65 columns):
RRS.S2502.Gunit.M1 G1.AVL            5237 non-null int64
YVIK-YtreVikna1.-G2-T4015A3 -0104    5222 non-null float64
RRS.S2502.Gunit.M2 G1.AVL            5237 non-null int64
YVIK-YtreVikna1.-G3-T4015A3 -0104    5222 non-null float64
RRS.S2502.Gunit.M3 G1.AVL            5237 non-null int64
YVIK-YtreVikna1.-G4-T4015A3 -0104    5222 non-null float64
RRS.S2502.Gunit.M4 G1.AVL            5237 non-null int64
YVIK-YtreVikna1.-G5-T4015A3 -0104    5222 non-null float64
RRS.S2502.Gunit.M5 G1.AVL            5237 non-null int64
YVIK-YtreVikna1.-G6-T4015A3 -0104    5222 non-null float64
RRS.S2502.Gunit.M6 G1.AVL            5237 non-null int64
YVIK-YtreVikna1.-G7-T4015A3 -0104    5222 non-null float64
RRS.S2502.Gunit.M7 G1.AVL            5237 non-null int64
YVIK-YtreVikna1.-G8-T4015A3 -0104    5222 non-null float64
RRS.S2502.Gunit.M8 G1.AVL            5237 non-null int64
YVIK-

In [3]:
data = data_raw.dropna()
data.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 5160 entries, 1.691883301 to 2.072566406
Data columns (total 65 columns):
RRS.S2502.Gunit.M1 G1.AVL            5160 non-null int64
YVIK-YtreVikna1.-G2-T4015A3 -0104    5160 non-null float64
RRS.S2502.Gunit.M2 G1.AVL            5160 non-null int64
YVIK-YtreVikna1.-G3-T4015A3 -0104    5160 non-null float64
RRS.S2502.Gunit.M3 G1.AVL            5160 non-null int64
YVIK-YtreVikna1.-G4-T4015A3 -0104    5160 non-null float64
RRS.S2502.Gunit.M4 G1.AVL            5160 non-null int64
YVIK-YtreVikna1.-G5-T4015A3 -0104    5160 non-null float64
RRS.S2502.Gunit.M5 G1.AVL            5160 non-null int64
YVIK-YtreVikna1.-G6-T4015A3 -0104    5160 non-null float64
RRS.S2502.Gunit.M6 G1.AVL            5160 non-null int64
YVIK-YtreVikna1.-G7-T4015A3 -0104    5160 non-null float64
RRS.S2502.Gunit.M7 G1.AVL            5160 non-null int64
YVIK-YtreVikna1.-G8-T4015A3 -0104    5160 non-null float64
RRS.S2502.Gunit.M8 G1.AVL            5160 non-null int64
YVIK-

In [4]:
num_features = len(data.columns) -1
print(num_features)

64


#### Konvertere til numpy-arrays

In [5]:
# Konverterer pandas.DataFrame til en numpy array
all_features_and_target_value = data.values.astype("float32")

# Trekker ut alle features (5 aromepunkter med 4 features + delayed = 21 features)
x = all_features_and_target_value[:,0:num_features]

# Trekker ut produksjonen som y-verdi
y = all_features_and_target_value[:,num_features]

#### Skalere data

In [6]:
from sklearn.preprocessing import MinMaxScaler

# Lage en scaler
scaler = MinMaxScaler()

# Anvende på features, her lagret i variablen x.
x = scaler.fit_transform(x)

#### Train/test Split

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state = 67, shuffle=False)

### Konvertere til tidserie matrise

In [8]:
# Definere variabler:
n_backward = 3

In [9]:
# Function for making time series
def make_time_series(row_features, look_back_num):
    list_of_matrices = []
    
    num_time_steps = look_back_num + 1
    
    i = num_time_steps
    
    while i < len(row_features):
        list_of_matrices.append(row_features[(i-num_time_steps):i,:])
        
        i = i + 1
    
    return list_of_matrices

In [10]:
x_train_matrix = np.asarray(make_time_series(X_train, n_backward))
x_test_matrix = np.asarray(make_time_series(X_test, n_backward))

In [11]:
x_train_matrix.shape

(3608, 4, 64)

In [12]:
x_train_matrix[0].shape

(4, 64)

In [13]:
x_train_matrix[0]

array([[ 1.        ,  0.01747815,  0.        ,  0.77792865,  1.        ,
         0.86680442,  1.        ,  0.87214816,  1.        ,  0.86295372,
         1.        ,  0.94334579,  1.        ,  0.93343031,  1.        ,
         0.        ,  1.        ,  0.92894208,  1.        ,  0.68720323,
         1.        ,  0.73323661,  1.        ,  0.37420234,  1.        ,
         0.63871813,  1.        ,  0.84074271,  1.        ,  0.90905797,
         1.        ,  0.7998963 ,  1.        ,  0.73661494,  0.34328359,
         0.3069464 ,  0.26829267,  0.19148017,  0.36723161,  0.32739758,
         0.47422677,  0.93036211,  0.23920847,  0.49230886,  0.50394714,
         0.62438965,  0.95576745,  0.22623329,  0.49417847,  0.61640549,
         0.95209366,  0.26585844,  0.49230886,  0.56917381,  0.95821154,
         0.23920847,  0.4806087 ,  0.58900356,  0.9543305 ,  0.22280048,
         0.4798933 ,  0.59886265,  0.95918965,  0.18142872],
       [ 1.        ,  0.01747815,  0.        ,  0.85387242,  1.

In [14]:
y_train

array([ 27.92919922,  18.84036446,  16.92293167, ...,   2.20245004,
         2.47471666,   2.31861663], dtype=float32)

#### Fjerne 'n_backward' antall rader, for å få like store sett

In [15]:
y_train_matrix = np.delete(y_train, [x for x in range(n_backward+1)], 0)
y_test_matrix = np.delete(y_test, [x for x in range(n_backward+1)], 0)

In [16]:
y_train_matrix.shape

(3608,)

# Keras

In [17]:
import tensorflow as tf

#### Importere modellen

In [18]:
from tensorflow.contrib.keras import models

In [19]:
dnn_keras_model = models.Sequential()

#### Sette opp layers

In [20]:
from tensorflow.contrib.keras import layers

In [21]:
# Input layer
dnn_keras_model.add(layers.LSTM(units=32, return_sequences = True, input_shape=(x_train_matrix.shape[1],x_train_matrix.shape[2]) ))

In [22]:
'''Hidden Layers'''
#dnn_keras_model.add(layers.LSTM(units=32, return_sequences = True))

dnn_keras_model.add(layers.LSTM(units=16))

In [23]:
# Output Layer
dnn_keras_model.add(layers.Dense(1))

#### Kompilere modellen

In [24]:
from tensorflow.contrib.keras import losses,optimizers,metrics,activations

In [25]:
#Kompilere - Alternative lossfunctions: mean_squared_error
dnn_keras_model.compile(optimizer='adam', loss = 'mean_absolute_error')

#### Trene modellen

In [26]:
from tensorflow.contrib.keras import losses,optimizers,metrics,activations

#### Callbacks og checkpoints

In [27]:
from tensorflow.contrib.keras import callbacks

In [28]:
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=500)

checkpoint = callbacks.ModelCheckpoint('checkpoint_model_LSTM_advanced.h5', monitor = 'val_loss', 
                                       verbose = 1, save_best_only= True, mode= 'min')

In [29]:
# Trene modellen
np.random.seed(7)
dnn_keras_model.fit(x_train_matrix,y_train_matrix, epochs = 1000, batch_size=32, verbose=2, 
                       validation_split=0.2,
                       callbacks=[checkpoint,early_stopping])


# validation_split=0.20
#validation_data=(X_test, y_test)

Train on 2886 samples, validate on 722 samples
Epoch 1/1000
Epoch 00000: val_loss improved from inf to 10.98663, saving model to checkpoint_model_LSTM_advanced.h5
2s - loss: 11.2825 - val_loss: 10.9866
Epoch 2/1000
Epoch 00001: val_loss improved from 10.98663 to 10.10072, saving model to checkpoint_model_LSTM_advanced.h5
0s - loss: 10.0451 - val_loss: 10.1007
Epoch 3/1000
Epoch 00002: val_loss improved from 10.10072 to 9.22760, saving model to checkpoint_model_LSTM_advanced.h5
0s - loss: 9.0874 - val_loss: 9.2276
Epoch 4/1000
Epoch 00003: val_loss improved from 9.22760 to 8.56588, saving model to checkpoint_model_LSTM_advanced.h5
0s - loss: 8.4220 - val_loss: 8.5659
Epoch 5/1000
Epoch 00004: val_loss improved from 8.56588 to 7.91994, saving model to checkpoint_model_LSTM_advanced.h5
0s - loss: 7.7991 - val_loss: 7.9199
Epoch 6/1000
Epoch 00005: val_loss improved from 7.91994 to 7.42158, saving model to checkpoint_model_LSTM_advanced.h5
0s - loss: 7.3204 - val_loss: 7.4216
Epoch 7/1000


Epoch 00066: val_loss did not improve
0s - loss: 2.2344 - val_loss: 3.1381
Epoch 68/1000
Epoch 00067: val_loss did not improve
0s - loss: 2.2615 - val_loss: 3.3553
Epoch 69/1000
Epoch 00068: val_loss did not improve
0s - loss: 2.1662 - val_loss: 3.0950
Epoch 70/1000
Epoch 00069: val_loss did not improve
0s - loss: 2.1820 - val_loss: 3.1745
Epoch 71/1000
Epoch 00070: val_loss did not improve
0s - loss: 2.2078 - val_loss: 3.1395
Epoch 72/1000
Epoch 00071: val_loss did not improve
0s - loss: 2.1953 - val_loss: 2.9782
Epoch 73/1000
Epoch 00072: val_loss improved from 2.96667 to 2.92440, saving model to checkpoint_model_LSTM_advanced.h5
0s - loss: 2.2060 - val_loss: 2.9244
Epoch 74/1000
Epoch 00073: val_loss did not improve
0s - loss: 2.1533 - val_loss: 3.2373
Epoch 75/1000
Epoch 00074: val_loss did not improve
0s - loss: 2.1428 - val_loss: 3.3704
Epoch 76/1000
Epoch 00075: val_loss did not improve
0s - loss: 2.1686 - val_loss: 3.4456
Epoch 77/1000
Epoch 00076: val_loss did not improve
0s -

KeyboardInterrupt: 

# Resultater

In [ ]:
from sklearn import metrics

In [ ]:
#finished_model = dnn_keras_model

finished_model = models.load_model('checkpoint_model_LSTM_advanced.h5')

#### Prediksjon på testsett

In [ ]:
final_preds = finished_model.predict(x_test_matrix)

In [ ]:
# 5000 epocs , b_size = 10, 24(24)[10](1)
print('Mean Absolute Error: \t\t\t', metrics.mean_absolute_error(y_test_matrix, final_preds))
print('Mean Squared Error: \t\t\t', metrics.mean_squared_error(y_test_matrix, final_preds))
print('Root Mean Squared Error: \t\t', np.sqrt(metrics.mean_squared_error(y_test_matrix, final_preds)))

#### Prediksjon på treningdata

In [ ]:
train_preds = finished_model.predict(x_train_matrix)

In [ ]:
# 70% av data (benyttet til treningen)
print('Mean Absolute Error: \t\t\t', metrics.mean_absolute_error(y_train_matrix,train_preds))
print('Mean Squared Error: \t\t\t', metrics.mean_squared_error(y_train_matrix, train_preds))
print('Root Mean Squared Error: \t\t', np.sqrt(metrics.mean_squared_error(y_train_matrix, train_preds)))

# Visualisere resultater

#### Lager oversikt over testdataen

In [ ]:
predictions = list(final_preds)

predictions_list = []

for pred in predictions:
    predictions_list.append(pred[0])

In [ ]:
Oversikt = pd.DataFrame(data = {'real': y_test_matrix, 'predicitions': predictions_list})

Oversikt['differanse'] = Oversikt['real'] - Oversikt['predicitions'] 

Oversikt['abs_diff'] = Oversikt['differanse'].apply(abs)

### Scatterplot med fargekodede prediskjoner

In [ ]:
ax = Oversikt.plot.scatter(x='real',y='predicitions',
                   c='abs_diff',cmap='coolwarm', figsize = (20,10))

ax.set_xlabel("x label")

#remove spines
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

fig = ax.get_figure()
#fig.savefig('keras_model.png')

### Feilfordelingen av prediksjoner

In [ ]:
ax = Oversikt['differanse'].hist(bins=60,figsize = (20,10))

plt.xlabel('Verdi av feilprediskjon')
plt.ylabel('Antall timer')
plt.title('Fordeling av feilprediskjoner')

#remove spines
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

fig = ax.get_figure()
#fig.savefig('hist av abs.png')

# Ekte plott av prediksjonene

#### Forsøk på å hente ut et utdrag av dataen

In [ ]:
all_predictions = finished_model.predict(x[:,:])

predictions_all = list(all_predictions)

predictions_all_list = []

for pred in predictions_all:
    predictions_all_list.append(pred[0])
    
timeline = pd.DataFrame(data = {'real': y[:], 'predicitions': predictions_all_list})

# Adjust in order to plot 'dagens modell'
data_raw.dropna(inplace=True)
data_raw.reset_index(inplace=True)

#### Tidsplot av data

In [ ]:
# Justere tidslinjen på plottet
start = 0
slutt = 20

ax = timeline.loc[start:slutt,'real'].plot(figsize=(20,8))
ax = timeline.loc[start:slutt,'predicitions'].plot(figsize=(20,8))
ax = data_raw.loc[start:slutt,'YVIK-YtreVikna1-Sum-produksjon'].plot(figsize=(20,8))

 
plt.xlabel('Tid (antall timer)')
plt.ylabel('Produksjon i Mega Watt (MW)')
plt.title('Utdrag fra tidsperioden')

plt.legend(loc='best')

#remove spines
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

# Saves figure
fig = ax.get_figure()
#fig.savefig('september10.png')